# Data exploration for joining all data sources

In this notebook we develop the functions for retrieving the tables, preparing them for the join, and then combining them based on the timestamp. 

## Imports

In [1]:
import os
import pandas as pd
import numpy as np
import csv
import datetime
import collections

## Sources
In the following section we construct the methods to read the different csv files and convert them to flat tables. 

In [2]:
location = "../data/"

In [3]:
class Helper():

    @classmethod
    def timestamp_to_datetimes(cls, df: pd.DataFrame): 
        """ Takes a pandas dataframe with a timestamp column (int) and also adds date datetime, datetime_ms, datetime_μs. 
        """
        # Conversion functions
        conversion_to_datetime_μs = lambda x: datetime.datetime.fromtimestamp(x/1000000000).strftime('%Y-%m-%d %H:%M:%S.%f')
        conversion_to_datetime_ms = lambda x: datetime.datetime.fromtimestamp(x/1000000000).strftime('%Y-%m-%d %H:%M:%S.%f')[:-3]
        conversion_to_datetime = lambda x: datetime.datetime.fromtimestamp(x/1000000000).strftime('%Y-%m-%d %H:%M:%S')

        # Apply conversions
        df["datetime_μs"] = df["timestamp"].apply(conversion_to_datetime_μs)
        df["datetime_ms"] = df["timestamp"].apply(conversion_to_datetime_ms)
        df["datetime"] = df["timestamp"].apply(conversion_to_datetime)

        return

### SSD2 Data
Data from the WE7000 DAQ for the SSD2. The PMT current from the MOT will be obtained like this in our next experiment.

In [4]:
class SSD(): 
    """ Class for data engineering of the SSD2 data. """
    
    @classmethod
    def get_table(cls, filepath: str=location+"-20220314-100806-Slot1-In2.csv", with_metadata: bool=False): 
        """ Load full table, consisting of data and metadata in a flat format. """
        data_df = cls._get_data(filepath)
        metadata_df = cls._get_metadata(filepath)
        df = data_df.merge(metadata_df, how='cross')
        cls._harmonize_time(df)
        df = df.sort_values(by="timestamp")
        if with_metadata: 
            return df
        return df[[col for col in df.columns if col not in metadata_df.columns]]

    @classmethod
    def _get_data(cls, filepath: str=location+"-20220314-100806-Slot1-In2.csv") -> pd.DataFrame: 
         return pd.read_csv(filepath_or_buffer=filepath, 
                                 skiprows=38, 
                                 header=0, 
                                 names=["TraceName", "Time", "PulseHeight"])
        
    @classmethod
    def _get_metadata(cls, filepath: str=location+"-20220314-100806-Slot1-In2.csv"): 
        with open(filepath, newline='') as f:
            reader = csv.reader(f)
            metadata = list(reader)[:38]
            metadata =  metadata[:3] +  metadata[4:]
            columns = [line[0] for line in metadata]
            row = [line[1] for line in metadata]
            df = pd.DataFrame(data=[row], columns=columns)
            return df
    
    @classmethod
    def _harmonize_time(cls, df: pd.DataFrame): 
        """ Convert the relative time and start time to the real time. """
        
        # Start time
        helper_df = pd.DataFrame()
        helper_df["start_datetime_str"] = df["Date"].apply(lambda s: s.replace("/", "-")) + " " + df["//StartTime"]
        helper_df["start_datetime"] = pd.to_datetime(helper_df["start_datetime_str"]) 
    
        # Conversion parameter: Time_x * rel_time_to_ns = rel. time in ns
        time_resolution = df['//TimeResolution'][0]
        rel_time_to_ns = {
            '1.000000e-009': 1e-0,
            '1.000000e-006': 1e+3,
            '1.000000e-003': 1e+6
        }[time_resolution]
        
        # Real time
        helper_df["relative_time_ns"] = df["Time_x"] * rel_time_to_ns
        helper_df["start_ns"] = helper_df.start_datetime.values.astype(np.int64)
        helper_df["timestamp"] = helper_df["start_ns"] + helper_df["relative_time_ns"]
        
        # Add datetimes
        df["timestamp"] = helper_df["timestamp"]
        Helper.timestamp_to_datetimes(df)
    
        return df

In [5]:
ssd_df = SSD.get_table(with_metadata=True)
ssd_df

,TraceName,Time_x,PulseHeight,Model,BlockNumber,BlockSize,VUnit,HResolution,HUnit,Date,...,//StopSource,//TimeResolution,//TimeStamp,//RealTime,//LiveTime,//DeadTime(%),timestamp,datetime_μs,datetime_ms,datetime
0,1,865236,2855,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647252e+18,2022-03-14 19:07:54.865236,2022-03-14 19:07:54.865,2022-03-14 19:07:54
1,2,2486115,2516,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647252e+18,2022-03-14 19:07:56.486115,2022-03-14 19:07:56.486,2022-03-14 19:07:56
2,3,2628986,2527,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647252e+18,2022-03-14 19:07:56.628986,2022-03-14 19:07:56.628,2022-03-14 19:07:56
3,4,2650471,2456,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647252e+18,2022-03-14 19:07:56.650471,2022-03-14 19:07:56.650,2022-03-14 19:07:56
4,5,3415610,2482,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647252e+18,2022-03-14 19:07:57.415610,2022-03-14 19:07:57.415,2022-03-14 19:07:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15256,15257,7133087349,2993,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647260e+18,2022-03-14 21:06:47.087349,2022-03-14 21:06:47.087,2022-03-14 21:06:47
15257,15258,7133923984,2465,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647260e+18,2022-03-14 21:06:47.923984,2022-03-14 21:06:47.923,2022-03-14 21:06:47
15258,15259,7134604904,3007,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647260e+18,2022-03-14 21:06:48.604904,2022-03-14 21:06:48.604,2022-03-14 21:06:48
15259,15260,7135583418,2726,WE7562,1,15262,cnt,1.000000e+000,?,2022/03/14,...,None,1.000000e-006,Peak,7.136569e+003,7.136410e+003,2.233544e-003,1.647260e+18,2022-03-14 21:06:49.583418,2022-03-14 21:06:49.583,2022-03-14 21:06:49


### SSD Histogram representation
A very useful representation of the SSD data is when we group by time (primary key) and then count the number of pulses with a certain hight. The columns then correspond to voltage ranges in 12-bit (0, 1, 2, 3, ..., 4094, 4095) encoding. 

In [6]:
class SSDRepresentation(): 
    
    @classmethod
    def get_hist_rep(cls, ssd_df: pd.DataFrame) -> pd.DataFrame: 
        nr_of_channels = ssd_df["//Gain"][0]
        
        # Groupby timestamp and PulseHeight (channel)
        grouped = ssd_df.groupby(by=["datetime", "PulseHeight"])
        df = grouped.agg({"TraceName": 'count'}).rename({"TraceName": "PulseCount"}, axis=1)
        df = df.unstack(level='PulseHeight').fillna(0, downcast="infer")
        
        # Fix column names
        df.columns = df.columns.to_flat_index()
        df.columns = [str(col[1]) for col in df.columns.values]
            
        return df
    
    @classmethod
    def get_fully_aggregated_hist_rep(cls, ssd_df: pd.DataFrame) -> pd.DataFrame: 
        df = cls.get_hist_rep(ssd_df) 
        df_sum = df.sum()
        df = pd.DataFrame(data=[df_sum.values], columns=df_sum.index.values)
        return df

In [7]:
ssd_hist_df = SSDRepresentation.get_hist_rep(ssd_df)
ssd_hist_df

,500,501,502,503,504,505,506,507,508,509,...,3543,3556,3661,3676,3719,3842,3929,3977,3984,4000
datetime,,,,,,,,,,,,,,,,,,,,,
2022-03-14 19:07:54,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-14 19:07:56,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-14 19:07:57,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-14 19:07:58,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-14 19:07:59,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14 21:06:45,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-14 21:06:47,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2022-03-14 21:06:48,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [8]:
ssd_full_agg_hist_df = SSDRepresentation.get_fully_aggregated_hist_rep(ssd_df)
ssd_full_agg_hist_df

,500,501,502,503,504,505,506,507,508,509,...,3543,3556,3661,3676,3719,3842,3929,3977,3984,4000
0,2,4,6,1,3,2,3,3,2,6,...,1,1,1,1,1,1,1,1,1,1


### PMT Data
In the table, we have the following columns. 
- No. : Frame number of CMOS camera
- Time: When the frame is obtained
- PMT Current: Current from photomultiplier at the time the frame obtained
- ROI Sum: The sum of signal values in ROI
- Coil (1:ON 0:ODD): The current of the coil at that time


In [9]:
class PMT(): 
    
    @classmethod
    def get_table(cls, filepath: str=location+"all_data.csv"): 
        df = pd.read_csv(filepath_or_buffer=filepath)
        df.rename({"Unnamed: 5": "a"}, axis="columns", inplace=True)
        df = df.drop(["a"], axis=1)
        cls._harmonize_time(df)
        df = df.sort_values(by="timestamp")
        return df
    
    @classmethod
    def _harmonize_time(cls, df: pd.DataFrame): 
        df["datetime_μs"] = df["Time"].apply(lambda s: s+"000")
        df["datetime_ms"] = df["Time"]
        df["datetime"] = df["Time"].apply(lambda s: s[:-4])
        df["timestamp"] = df["Time"].apply(pd.Timestamp).values.astype(np.int64)
        return 
    
pmt_df = PMT.get_table()
pmt_df

,No.,Time,PMT Current (A),ROI Sum,Coil (1:ON 0:OFF),datetime_μs,datetime_ms,datetime,timestamp
0,1,2022/03/15 08:18:00.266,-1.519308e-13,1491678,1,2022/03/15 08:18:00.266000,2022/03/15 08:18:00.266,2022/03/15 08:18:00,1647332280266000000
1,14,2022/03/15 08:18:01.507,9.085866e-13,1498276,1,2022/03/15 08:18:01.507000,2022/03/15 08:18:01.507,2022/03/15 08:18:01,1647332281507000000
2,18,2022/03/15 08:18:01.907,3.832096e-13,1499445,1,2022/03/15 08:18:01.907000,2022/03/15 08:18:01.907,2022/03/15 08:18:01,1647332281907000000
3,22,2022/03/15 08:18:02.308,3.028373e-15,1499770,1,2022/03/15 08:18:02.308000,2022/03/15 08:18:02.308,2022/03/15 08:18:02,1647332282308000000
4,26,2022/03/15 08:18:02.710,4.986230e-14,1498276,1,2022/03/15 08:18:02.710000,2022/03/15 08:18:02.710,2022/03/15 08:18:02,1647332282710000000
...,...,...,...,...,...,...,...,...,...
114062,457337,2022/03/15 21:00:57.162,-8.295666e-06,1491353,1,2022/03/15 21:00:57.162000,2022/03/15 21:00:57.162,2022/03/15 21:00:57,1647378057162000000
114063,457341,2022/03/15 21:00:57.563,-8.267808e-06,1491303,1,2022/03/15 21:00:57.563000,2022/03/15 21:00:57.563,2022/03/15 21:00:57,1647378057563000000
114064,457345,2022/03/15 21:00:57.963,-8.278364e-06,1491668,1,2022/03/15 21:00:57.963000,2022/03/15 21:00:57.963,2022/03/15 21:00:57,1647378057963000000
114065,457349,2022/03/15 21:00:58.362,-8.336747e-06,1491360,1,2022/03/15 21:00:58.362000,2022/03/15 21:00:58.362,2022/03/15 21:00:58,1647378058362000000


### Coil Log
The current of the MOT coil is controlled by a relay switch. This text file is the log of the relay switch.

In [10]:
class Coil(): 
    
    @classmethod
    def get_table(cls, filepath: str=location+"coil_log.txt"): 
        df = pd.read_csv(filepath_or_buffer=filepath, delimiter="	")
        cls._harmonize_time(df)
        df = df.sort_values(by="timestamp")
        return df
    
    @classmethod
    def _harmonize_time(cls, df: pd.DataFrame): 
        df["datetime_μs"] = df["Time"].apply(lambda s: s+"000")
        df["datetime_ms"] = df["Time"]
        df["datetime"] = df["Time"].apply(lambda s: s[:-4])
        df["timestamp"] = df["Time"].apply(pd.Timestamp).values.astype(np.int64)
        return 

coil_table = Coil.get_table()
coil_table 

,Time,CoilOperation,datetime_μs,datetime_ms,datetime,timestamp
0,2022/03/14 12:09:23,ON,2022/03/14 12:09:23000,2022/03/14 12:09:23,2022/03/14 12:0,1647259763000000000
1,2022/03/14 12:09:23,OFF,2022/03/14 12:09:23000,2022/03/14 12:09:23,2022/03/14 12:0,1647259763000000000
2,2022/03/14 12:09:31,ON,2022/03/14 12:09:31000,2022/03/14 12:09:31,2022/03/14 12:0,1647259771000000000
3,2022/03/14 12:10:53,OFF,2022/03/14 12:10:53000,2022/03/14 12:10:53,2022/03/14 12:1,1647259853000000000
4,2022/03/14 12:12:57,OFF,2022/03/14 12:12:57000,2022/03/14 12:12:57,2022/03/14 12:1,1647259977000000000
5,2022/03/14 12:12:57,ON,2022/03/14 12:12:57000,2022/03/14 12:12:57,2022/03/14 12:1,1647259977000000000
6,2022/03/14 12:12:58,ON,2022/03/14 12:12:58000,2022/03/14 12:12:58,2022/03/14 12:1,1647259978000000000
7,2022/03/14 14:07:42,OFF,2022/03/14 14:07:42000,2022/03/14 14:07:42,2022/03/14 14:0,1647266862000000000
8,2022/03/14 14:07:45,ON,2022/03/14 14:07:45000,2022/03/14 14:07:45,2022/03/14 14:0,1647266865000000000
9,2022/03/14 15:40:46,OFF,2022/03/14 15:40:46000,2022/03/14 15:40:46,2022/03/14 15:4,1647272446000000000


### Heater Log
Log of the IR heater output percentage for target heating. 

In [11]:
class Heater(): 
    """ Class for data engineering of the heater data. """
    
    @classmethod
    def get_table(cls, filepath: str=location+"HeaterLog_20220314_100740_00001.csv", with_metadata: bool=False): 
        """ Load full table, consisting of data and metadata in a flat format. """
        data_df = cls._get_data(filepath)
        metadata_df = cls._get_metadata(filepath)
        df = data_df.merge(metadata_df, how='cross')
        cls._harmonize_time(df)
        if with_metadata: 
            return df
        return df[[col for col in df.columns if col not in metadata_df.columns]]    
    
    @classmethod
    def _get_data(cls, filepath: str=location+"HeaterLog_20220314_100740_00001.csv") -> pd.DataFrame: 
        df = pd.read_csv(filepath_or_buffer=filepath, 
                                 skiprows=6, 
                                 header=0, 
                                 names=["Date", "Time", "Unknown", "TargetPercentage", "MeasuredPercentage"],
                                 encoding="cp932")
        cls._harmonize_time(df)
        df = df.sort_values(by="timestamp")
        return df
    
    @classmethod
    def _get_metadata(cls, filepath: str=location+"HeaterLog_20220314_100740_00001.csv"): 
        with open(filepath, newline='', encoding="cp932") as f:
            reader = csv.reader(f)
            metadata_list = list(reader)[:6]
            columns = [m[0] for m in metadata_list]
            row = [metadata_list[i][1] for i in range(2)] +  [f"{metadata_list[i][3]},{metadata_list[i][4]}" for i in range(2, 6)]
            df = pd.DataFrame(data=[row], columns=columns)
            return df
    
    @classmethod
    def _harmonize_time(cls, df: pd.DataFrame) -> pd.DataFrame: 
        df["datetime"] = df["Date"].apply(lambda s: s.replace("/", "-")) + " " + df["Time"]
        df["datetime_μs"] = df["datetime"].apply(lambda s: s+".000000")
        df["datetime_ms"] = df["datetime"].apply(lambda s: s+".000")
        df["timestamp"] = df["datetime_ms"].apply(pd.Timestamp).values.astype(np.int64)
        return df

In [12]:
heater_df = Heater.get_table(with_metadata=False)
heater_df

,Date,Time,Unknown,TargetPercentage,MeasuredPercentage,datetime,datetime_μs,datetime_ms,timestamp
0,2022/03/14,10:07:42,0,0,0,2022-03-14 10:07:42,2022-03-14 10:07:42.000000,2022-03-14 10:07:42.000,1647252462000000000
1,2022/03/14,10:07:43,0,0,0,2022-03-14 10:07:43,2022-03-14 10:07:43.000000,2022-03-14 10:07:43.000,1647252463000000000
2,2022/03/14,10:07:44,0,0,0,2022-03-14 10:07:44,2022-03-14 10:07:44.000000,2022-03-14 10:07:44.000,1647252464000000000
3,2022/03/14,10:07:45,0,0,0,2022-03-14 10:07:45,2022-03-14 10:07:45.000000,2022-03-14 10:07:45.000,1647252465000000000
4,2022/03/14,10:07:46,0,0,0,2022-03-14 10:07:46,2022-03-14 10:07:46.000000,2022-03-14 10:07:46.000,1647252466000000000
...,...,...,...,...,...,...,...,...,...
59994,2022/03/15,02:47:43,0,0,0,2022-03-15 02:47:43,2022-03-15 02:47:43.000000,2022-03-15 02:47:43.000,1647312463000000000
59995,2022/03/15,02:47:44,0,0,0,2022-03-15 02:47:44,2022-03-15 02:47:44.000000,2022-03-15 02:47:44.000,1647312464000000000
59996,2022/03/15,02:47:45,0,0,0,2022-03-15 02:47:45,2022-03-15 02:47:45.000000,2022-03-15 02:47:45.000,1647312465000000000
59997,2022/03/15,02:47:46,0,0,0,2022-03-15 02:47:46,2022-03-15 02:47:46.000000,2022-03-15 02:47:46.000,1647312466000000000


### Ion Beam Control
Log of the Fr ion source. First day of experiment, right after the end of primary beam check, just starting the Fr ion extraction. The column "FC" is the current from either one of the faraday cups, or the sum of both.
The columns "Center" and "Surrounding" are the voltages applied to the mechanical relay switches that connects the faraday cups to the picoammeter. For example, if "Center" = 24 and "Surrounding" = 0, the value at "FC" is the current observed on FC Center in nA.

In [13]:
class IonBeamControl(): 
    
    @classmethod
    def get_table(cls, filepath: str=location+"IonBeamControl1.5_DESKTOP-8ICG2TJ_20220314_114132.csv"): 
        df = pd.read_csv(filepath_or_buffer=filepath)
        cls._harmonize_time(df)
        df = df.sort_values(by="timestamp")
        return df
    
    @classmethod
    def _harmonize_time(cls, df: pd.DataFrame) -> pd.DataFrame: 
        df["datetime"] = pd.to_datetime(df["Timestamp"])
        df["datetime_μs"] = df["Timestamp"].apply(lambda s: s+".000000")
        df["datetime_ms"] = df["Timestamp"].apply(lambda s: s+".000")
        df["timestamp"] = df["datetime"].values.astype(np.int64)
        
ion_beam_control_df = IonBeamControl.get_table()
ion_beam_control_df

,Timestamp,Be Bypass,Fr Beamline,Center,Surrounding,FC,NC,Be temp (degC),He press (V),Au temp (V),...,Einzel Current,Drift 2 Current,Freq (kHz),Phase (deg),Amp(CH1) (Vpp),Amp(CH2) (Vpp),datetime,datetime_μs,datetime_ms,timestamp
0,2022/03/14 11:41:38,0.00201,0.000044,0.0,0.0,-0.000301,0.0,0.00005,3.0185,0.71115,...,-2.800000e-07,4.000000e-08,0,0,0,0,2022-03-14 11:41:38,2022/03/14 11:41:38.000000,2022/03/14 11:41:38.000,1647258098000000000
1,2022/03/14 11:41:39,0.00201,0.000044,0.0,0.0,0.000835,0.0,0.00005,3.0175,0.71115,...,-2.700000e-07,4.000000e-08,0,0,0,0,2022-03-14 11:41:39,2022/03/14 11:41:39.000000,2022/03/14 11:41:39.000,1647258099000000000
2,2022/03/14 11:41:40,0.00201,0.000044,0.0,0.0,0.000641,0.0,0.00005,3.0175,0.71115,...,-2.700000e-07,4.000000e-08,0,0,0,0,2022-03-14 11:41:40,2022/03/14 11:41:40.000000,2022/03/14 11:41:40.000,1647258100000000000
3,2022/03/14 11:41:41,0.00201,0.000043,0.0,0.0,0.001132,0.0,0.00005,3.0175,0.71115,...,-2.700000e-07,3.000000e-08,0,0,0,0,2022-03-14 11:41:41,2022/03/14 11:41:41.000000,2022/03/14 11:41:41.000,1647258101000000000
4,2022/03/14 11:41:42,0.00200,0.000043,0.0,0.0,0.000448,0.0,0.00005,3.0180,0.71115,...,-2.700000e-07,3.000000e-08,0,0,0,0,2022-03-14 11:41:42,2022/03/14 11:41:42.000000,2022/03/14 11:41:42.000,1647258102000000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
637,2022/03/14 11:52:16,0.00160,0.000008,0.0,0.0,0.000062,0.0,0.00010,3.0195,0.71115,...,-2.800000e-07,3.000000e-08,0,0,0,0,2022-03-14 11:52:16,2022/03/14 11:52:16.000000,2022/03/14 11:52:16.000,1647258736000000000
638,2022/03/14 11:52:17,0.00160,0.000008,0.0,0.0,-0.000084,0.0,0.00010,3.0190,0.71110,...,-2.700000e-07,3.000000e-08,0,0,0,0,2022-03-14 11:52:17,2022/03/14 11:52:17.000000,2022/03/14 11:52:17.000,1647258737000000000
639,2022/03/14 11:52:18,0.00160,0.000008,0.0,0.0,0.000205,0.0,0.00010,3.0190,0.71110,...,-2.700000e-07,3.000000e-08,0,0,0,0,2022-03-14 11:52:18,2022/03/14 11:52:18.000000,2022/03/14 11:52:18.000,1647258738000000000
640,2022/03/14 11:52:19,0.00160,0.000008,0.0,0.0,0.000035,0.0,0.00010,3.0190,0.71110,...,-2.800000e-07,3.000000e-08,0,0,0,0,2022-03-14 11:52:19,2022/03/14 11:52:19.000000,2022/03/14 11:52:19.000,1647258739000000000


### Gauge Monitor

In [14]:
class Gauge(): 
    
    @classmethod
    def get_table(cls, filepath: str=location+"TPG256GaugeMonitor_Single_DESKTOP-BEF5FI4_20220312_203214.csv"): 
        df = pd.read_csv(filepath_or_buffer=filepath)
        cls._harmonize_time(df)
        df = df.sort_values(by="timestamp")
        return df
    
    @classmethod
    def _harmonize_time(cls, df: pd.DataFrame): 
        df["datetime"] = pd.to_datetime(df["Timestamp"])
        df["datetime_μs"] = df["Timestamp"].apply(lambda s: s+".000000")
        df["datetime_ms"] = df["Timestamp"].apply(lambda s: s+".000")
        df["timestamp"] = df["datetime"].values.astype(np.int64)
        return df

gauge_df = Gauge.get_table()
gauge_df

,Timestamp,Rb disp.,Neut.,Surf. Ref.,NC,NC.1,NC.2,datetime,datetime_μs,datetime_ms,timestamp
0,2022/03/12 20:32:19,8.950000e-07,1.342000e-06,8.270000e-07,1.000000e-09,5.009000e-02,0.05009,2022-03-12 20:32:19,2022/03/12 20:32:19.000000,2022/03/12 20:32:19.000,1647117139000000000
1,2022/03/12 20:32:20,8.930000e-07,1.334000e-06,8.260000e-07,1.000000e-09,5.009000e-02,0.05009,2022-03-12 20:32:20,2022/03/12 20:32:20.000000,2022/03/12 20:32:20.000,1647117140000000000
2,2022/03/12 20:32:21,8.910000e-07,1.336000e-06,8.280000e-07,1.000000e-09,5.009000e-02,0.05009,2022-03-12 20:32:21,2022/03/12 20:32:21.000000,2022/03/12 20:32:21.000,1647117141000000000
3,2022/03/12 20:32:22,8.950000e-07,1.337000e-06,8.280000e-07,2.000000e-07,5.009000e-02,0.05009,2022-03-12 20:32:22,2022/03/12 20:32:22.000000,2022/03/12 20:32:22.000,1647117142000000000
4,2022/03/12 20:32:23,8.940000e-07,1.336000e-06,8.270000e-07,2.000000e-07,5.009000e-02,0.05009,2022-03-12 20:32:23,2022/03/12 20:32:23.000000,2022/03/12 20:32:23.000,1647117143000000000
...,...,...,...,...,...,...,...,...,...,...,...
264358,2022/03/15 21:59:39,5.010000e-07,5.000000e-07,5.000000e-07,1.000000e-09,5.009000e-02,0.05009,2022-03-15 21:59:39,2022/03/15 21:59:39.000000,2022/03/15 21:59:39.000,1647381579000000000
264359,2022/03/15 21:59:40,5.000000e-07,5.000000e-07,5.000000e-07,2.000000e-07,5.009000e-02,0.05009,2022-03-15 21:59:40,2022/03/15 21:59:40.000000,2022/03/15 21:59:40.000,1647381580000000000
264360,2022/03/15 21:59:41,5.000000e-07,5.000000e-07,5.000000e-07,2.000000e-07,5.009000e-02,0.05009,2022-03-15 21:59:41,2022/03/15 21:59:41.000000,2022/03/15 21:59:41.000,1647381581000000000
264361,2022/03/15 21:59:42,5.000000e-07,5.000000e-07,5.000000e-07,1.000000e-09,2.000000e-07,0.05009,2022-03-15 21:59:42,2022/03/15 21:59:42.000000,2022/03/15 21:59:42.000,1647381582000000000


### Laser data

In [15]:
class Laser(): 
    
    @classmethod
    def get_table(cls, filepath: str=location+"15.03.2022, 21.30, 384.22817013 THz.lta", with_metadata: bool=False): 
        """ Load full table, consisting of data and metadata in a flat format. """
        data_df = cls._get_data(filepath)
        metadata_df = cls._get_metadata(filepath)
        if not with_metadata: 
            metadata_df = metadata_df[["StartTime"]]
        df = data_df.merge(metadata_df, how='cross')
        cls._harmonize_time(df)
        df = df.sort_values(by="timestamp")
        if with_metadata: 
            return df
        else:
            return df[[col for col in df.columns if col not in metadata_df.columns]]     
    
    @classmethod
    def _get_data(cls, filepath: str=location+"15.03.2022, 21.30, 384.22817013 THz.lta") -> pd.DataFrame:
        """ Loads the data and makes sure that each row has all laser measurements by aggregating rows. """
        original_df = pd.read_csv(filepath_or_buffer=filepath, 
                                  skiprows=119,
                                  delimiter="	")
        
        df = cls._aggregate_laser_rows(original_df)
        return df
        
    @classmethod
    def _aggregate_laser_rows(cls, original_df: pd.DataFrame): 
        """ Originally, one measurement of the six laser wavelengths is distributed over six rows. We aggregate 
            these rows into one row. The only tradeoff is that we have to approximate the time with the time
            of the last measurement. 
        """
        
        n = len(original_df.index)
        
        time_column = 'Time  [ms]'
        laser_columns = original_df.columns[1:]
        n_laser_columns = len(laser_columns)
        
        laser_column_lookup = {col: i for i, col in enumerate(laser_columns)}
        data_lookup = collections.defaultdict(list)
        row_lookup = {}
        row_list = []
        
        for index, row in original_df.iterrows():
            column_index = pd.Series.first_valid_index(row[1:])
            row_lookup[column_index] = row[column_index]
            # Count if all 6 lasers have been measured
            if len(row_lookup) == 6:     
                item = [row[time_column]] + [row_lookup[col] for col in laser_columns]
                row_list.append(item)
                row_lookup = {}

        df = pd.DataFrame(data=row_list, columns=original_df.columns)
        return df
        
    @classmethod
    def _get_metadata(cls, filepath: str=location+"15.03.2022, 21.30, 384.22817013 THz.lta"): 
        with open(filepath, newline='', encoding="cp932") as f:
            reader = csv.reader(f, delimiter="	")
            metadata_list = list(reader)[:119]
            
            # Title
            title_column = ["Title"]
            title_row = [metadata_list[0][0]]
            
            # General info
            gi_columns = [m[0] for m in metadata_list[1:7]]
            gi_rows = [cls._combine(m[1:]) for m in metadata_list[1:7]]
            
            # General settings
            gs_columns = [m[0] for m in metadata_list[9:20]]
            gs_rows = [cls._combine(m[1:]) for m in metadata_list[9:20]]

            # Frames 1-6
            frame_columns = (
                [m[0] for m in metadata_list[22:36]]
                + [m[0] for m in metadata_list[38:52]]
                + [m[0] for m in metadata_list[54:68]]
                + [m[0] for m in metadata_list[70:84]]
                + [m[0] for m in metadata_list[86:100]]
                + [m[0] for m in metadata_list[102:116]]
            )
            
            frame_rows = (
                [cls._combine(m[1:]) for m in metadata_list[22:36]]
                + [cls._combine(m[1:]) for m in metadata_list[38:52]]
                + [cls._combine(m[1:]) for m in metadata_list[54:68]]
                + [cls._combine(m[1:]) for m in metadata_list[70:84]]
                + [cls._combine(m[1:]) for m in metadata_list[86:100]]
                + [cls._combine(m[1:]) for m in metadata_list[102:116]]
            )
            
            columns = title_column + gi_columns + gs_columns + frame_columns
            row = title_row + gi_rows + gs_rows + frame_rows
            df = pd.DataFrame(data=[row], columns=columns)
            return df                                    
    
    @classmethod
    def _combine(cls, entries: list): 
        """ If entries has length 1, then it returns the entry. 
            Otherwise, it converts the list to a comma separated string. 
        """
        if len(entries) == 0: 
            return None
        
        if len(entries) == 1: 
            return entries[0]
        
        return ",".join(entries)
    
    @classmethod
    def _harmonize_time(cls, df: pd.DataFrame):

        # Convert 15.03.2022, 08:46:39.387 to 15-03-2022 08:46:39.387
        helper_df = pd.DataFrame()
        helper_df["StartTime"] = df["StartTime"].apply(lambda s: s.replace(",", "").replace(".", "-", 2))
        
        # Calculate absolute time based on relative time
        helper_df["start_datetime"] = pd.to_datetime(helper_df["StartTime"])
        helper_df["start_timestamp"] = helper_df["start_datetime"].values.astype(np.int64)
        helper_df["timestamp"] = helper_df["start_timestamp"] + df["Time  [ms]"] * 1e3
        
        # Add datetimes
        df["timestamp"] = helper_df["timestamp"]
        Helper.timestamp_to_datetimes(df)
        
        return

In [16]:
laser_df = Laser.get_table(with_metadata=False)
laser_df

,Time [ms],"Signal 8 Wavelength, vac. [nm]","Signal 1 Wavelength, vac. [nm]","Signal 4 Wavelength, vac. [nm]","Signal 6 Wavelength, vac. [nm]","Signal 3 Wavelength, vac. [nm]","Signal 2 Wavelength, vac. [nm]",timestamp,datetime_μs,datetime_ms,datetime
0,4.937473e+02,780.245909,780.246040,794.969235,718.216332,817.098723,718.136908,1.647334e+18,2022-03-15 17:46:39.387494,2022-03-15 17:46:39.387,2022-03-15 17:46:39
1,9.876469e+02,780.245909,780.246041,794.969150,718.216331,817.098719,718.136910,1.647334e+18,2022-03-15 17:46:39.387988,2022-03-15 17:46:39.387,2022-03-15 17:46:39
2,1.481494e+03,780.245909,780.246040,794.969186,718.216333,817.098724,718.136907,1.647334e+18,2022-03-15 17:46:39.388482,2022-03-15 17:46:39.388,2022-03-15 17:46:39
3,1.975374e+03,780.245909,780.246039,794.969171,718.216329,817.098724,718.136915,1.647334e+18,2022-03-15 17:46:39.388975,2022-03-15 17:46:39.388,2022-03-15 17:46:39
4,2.469241e+03,780.245909,780.246041,794.969131,718.216332,817.098719,718.136905,1.647334e+18,2022-03-15 17:46:39.389469,2022-03-15 17:46:39.389,2022-03-15 17:46:39
...,...,...,...,...,...,...,...,...,...,...,...
78851,4.403330e+07,780.245909,780.246040,794.969182,718.281473,817.098739,718.136901,1.647334e+18,2022-03-15 17:47:23.420296,2022-03-15 17:47:23.420,2022-03-15 17:47:23
78852,4.403372e+07,780.245909,780.246042,794.969179,718.282609,817.098734,718.136919,1.647334e+18,2022-03-15 17:47:23.420725,2022-03-15 17:47:23.420,2022-03-15 17:47:23
78853,4.403415e+07,780.245909,780.246038,794.969174,718.282485,817.098738,718.136892,1.647334e+18,2022-03-15 17:47:23.421152,2022-03-15 17:47:23.421,2022-03-15 17:47:23
78854,4.403471e+07,780.245909,780.246043,794.969246,718.282392,817.098735,718.136906,1.647334e+18,2022-03-15 17:47:23.421715,2022-03-15 17:47:23.421,2022-03-15 17:47:23


### Image data
We have a folder which contains the picture data from the CMOS camera. A part of the pixels (region of interest, ROI) selected in advance is extracted. Each picture data is a csv file and records signals from each pixel.

In [17]:
class Image(): 
    
    def get_array(filepath: str=location+"cmos_000039.csv"): 
        with open(filepath) as file_name:
            return np.loadtxt(file_name, delimiter=",")
        
    def get_metadata(filepath: str=location+"cmos_000039.csv") -> pd.DataFrame: 
        
        columns = ["size", "ctime"] # Size and creation time
        row = [os.path.getsize(filepath), os.path.getctime(filepath)]
        return pd.DataFrame(data=[row], columns=columns)
        
image_array = Image.get_array()
image_table = Image.get_metadata()
image_table

,size,ctime
0,168214,1.661918e+09


In [18]:
image_array

array([[1., 1., 1., ..., 1., 1., 2.],
       [1., 2., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 2., 1., 1.]])

## Joins
For a refresher on joins, checkout this article: https://pandas.pydata.org/docs/user_guide/merging.html. Now our goal is to join the tables into a main table. We join on the timestamp and define the following rules: 
- We join on timestamp in seconds with an outer join.
- As the values do not exactly match, we first sort the dataframes, such that we then can join on nearly-matching values with some threshold (https://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.merge_asof.html). 

Problem: 
- As the sampling method varies a lot, we would lose a lot of data with this method. 

Solution A: 
- Treat some tables as parameters and settings, and some tables as measurements. Then we first sample the parameters and settings on level second, and then use a finer sampling for the measurements. 
- For this, we first find all unique times in seconds. Then we generate the rows for each unique second, taking the value if it exists or nan if not. 
- Last, we do an outer join on the time in seconds. 

In [19]:
class Main(): 

    def limit_tables_to_timespan(dfs, start, stop): 
        """ Takes a list of dfs, only keeps the rows between start and stop time and returns the modified list. 
        """
        pass
    
    def build_table(dfs: list, prefixes: list): 
        """ Concatenates the dataframes to a single dataframe, ignoring the indices. The names are used as prefixes of the
            columns, such that we can have similar column names but still know from which df it came. 
        """   
          
        # Add prefixes
        dfs = [df.rename(columns={col: prefix+"_"+col for col in df.columns}) for df, prefix in zip(dfs, prefixes)]
        
        # Outer join 
        main_df = pd.concat(dfs, axis=1, join="outer")
        
        return main_df

In [20]:
main_table = Main.build_table(
    dfs=[laser_df, gauge_df, ion_beam_control_df, heater_df, coil_table, pmt_df, ssd_df, ssd_hist_df],
    prefixes=["laser", "gauge", "ion_beam_control", "heater", "coil", "pmt", "ssd", "ssd_hist"]
)
main_table

,laser_Time [ms],"laser_Signal 8 Wavelength, vac. [nm]","laser_Signal 1 Wavelength, vac. [nm]","laser_Signal 4 Wavelength, vac. [nm]","laser_Signal 6 Wavelength, vac. [nm]","laser_Signal 3 Wavelength, vac. [nm]","laser_Signal 2 Wavelength, vac. [nm]",laser_timestamp,laser_datetime_μs,laser_datetime_ms,...,ssd_hist_3543,ssd_hist_3556,ssd_hist_3661,ssd_hist_3676,ssd_hist_3719,ssd_hist_3842,ssd_hist_3929,ssd_hist_3977,ssd_hist_3984,ssd_hist_4000
0,493.7473,780.245909,780.246040,794.969235,718.216332,817.098723,718.136908,1.647334e+18,2022-03-15 17:46:39.387494,2022-03-15 17:46:39.387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,987.6469,780.245909,780.246041,794.969150,718.216331,817.098719,718.136910,1.647334e+18,2022-03-15 17:46:39.387988,2022-03-15 17:46:39.387,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1481.4935,780.245909,780.246040,794.969186,718.216333,817.098724,718.136907,1.647334e+18,2022-03-15 17:46:39.388482,2022-03-15 17:46:39.388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1975.3737,780.245909,780.246039,794.969171,718.216329,817.098724,718.136915,1.647334e+18,2022-03-15 17:46:39.388975,2022-03-15 17:46:39.388,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2469.2408,780.245909,780.246041,794.969131,718.216332,817.098719,718.136905,1.647334e+18,2022-03-15 17:46:39.389469,2022-03-15 17:46:39.389,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-03-14 21:06:45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-14 21:06:47,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-14 21:06:48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2022-03-14 21:06:49,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Analysis 
In the following we want to test some simple analysis as proof of concept.